In [1]:
pip install pulp

Note: you may need to restart the kernel to use updated packages.


In [2]:
 import pandas as pd
 import matplotlib.pyplot as plt
 import numpy as np
 import math
 import pulp
 import warnings
 warnings.filterwarnings('ignore')
 pd.set_option('display.max_columns',None)
 pd.set_option('display.expand_frame_repr',False)

In [3]:
df1=pd.read_csv('ind_sl_t20i_1st_2022_02_24.csv')
df1.head()

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,PlayingXI,Points,DT
0,3692,38632,I Kishan,10276,IND,2,WK,73.23,8.5,0,1,141,1
1,3692,38632,S Iyer,9428,IND,2,BAT,71.23,8.5,0,1,84,1
2,3692,38632,B Kumar,1726,IND,2,BOWL,22.17,8.5,0,1,76,1
3,3692,38632,C Asalanka,10934,SL,5,BAT,64.60,8.5,0,1,70,1
4,3692,38632,R Sharma,576,IND,2,BAT,87.30,10.5,0,1,66,1


In [4]:
def fantasy(df):
    df=df[['Player','Team','Credits','Points','Role','PlayingXI']]
    df=df[df['PlayingXI']==1].reset_index(drop=True)
    df=pd.get_dummies(data=df,columns=['Role','Team'],dtype='int')
 #define decision variable
    pickup_status=pulp.LpVariable.dicts('pickup_status',((obj) for obj in df.index), cat='Binary')
 #define objective function
    m=pulp.LpProblem('Points maximisation problem',pulp.LpMaximize)
    m+=pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Points'] for obj in df.index])
 #define constraints
    m+=pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Credits']] for obj in df.index) <=100
 #11 players
    m+=pulp.lpSum([pickup_status[obj]*df.loc[(obj),'PlayingXI']] for obj in df.index) ==11
 #team constraints
    m+=pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Team_IND']] for obj in df.index) >=4
    m+=pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Team_IND']] for obj in df.index) <=7
    m+=pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Team_SL']] for obj in df.index) >=4
    m+=pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Team_SL']] for obj in df.index) <=7
 #role constraints
    m+=pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Role_WK']] for obj in df.index) >=1
    m+=pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Role_WK']] for obj in df.index) <=4
    m+=pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Role_BAT']] for obj in df.index) >=3
    m+=pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Role_BAT']] for obj in df.index) <=6
    m+=pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Role_BOWL']] for obj in df.index) >=3
    m+=pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Role_BOWL']] for obj in df.index) <=6
    m+=pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Role_AR']] for obj in df.index) >=1
    m+=pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Role_AR']] for obj in df.index) <=4
    m.solve()
    df['pickup_status']=0
    for obj in df.index:
        if pickup_status[obj].varValue:
            df['pickup_status'][obj]=1
    fdf=df[df['pickup_status']==1].sort_values(['Role_BAT','Role_WK','Role_AR','Role_BOWL'],ascending=False).reset_index(drop=True)

    return fdf

In [5]:
f_df1=fantasy(df1)

In [6]:
f_df1

,Player,Credits,Points,PlayingXI,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL,pickup_status
0,S Iyer,8.5,84,1,0,1,0,0,1,0,1
1,C Asalanka,8.5,70,1,0,1,0,0,0,1,1
2,R Sharma,10.5,66,1,0,1,0,0,1,0,1
3,I Kishan,8.5,141,1,0,0,0,1,1,0,1
4,V Iyer,8.5,50,1,1,0,0,0,1,0,1
5,R Jadeja,9.0,34,1,1,0,0,0,1,0,1
6,D Shanaka,9.0,32,1,1,0,0,0,0,1,1
7,B Kumar,8.5,76,1,0,0,1,0,1,0,1
8,Y Chahal,8.5,35,1,0,0,1,0,1,0,1
9,L Kumara,8.5,35,1,0,0,1,0,0,1,1


In [7]:
sum(f_df1.Credits)

97.0

In [10]:
p1 = sum(f_df1.Points)

In [12]:
df2 = pd.read_csv('ind_sl_t20i_2nd_2022_02_26.csv')
df2.head()

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points,DT
0,3692,38637,S Iyer,9428,IND,2,BAT,87.64,8.5,84,1,1,112,1
1,3692,38637,P Nissanka,13682,SL,5,BAT,66.77,9.5,2,0,1,100,1
2,3692,38637,R Jadeja,587,IND,2,AR,76.24,9.0,34,1,1,93,1
3,3692,38637,D Shanaka,8422,SL,5,AR,68.26,9.0,32,1,1,77,1
4,3692,38637,S Samson,8271,IND,2,WK,25.94,8.0,0,0,1,59,1


In [14]:
f_df2=fantasy(df2)
f_df2

,Player,Credits,Points,PlayingXI,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL,pickup_status
0,S Iyer,8.5,112,1,0,1,0,0,1,0,1
1,P Nissanka,9.5,100,1,0,1,0,0,0,1,1
2,D Gunathilaka,8.5,56,1,0,1,0,0,0,1,1
3,S Samson,8.0,59,1,0,0,0,1,1,0,1
4,R Jadeja,9.0,93,1,1,0,0,0,1,0,1
5,D Shanaka,9.0,77,1,1,0,0,0,0,1,1
6,L Kumara,8.5,52,1,0,0,1,0,0,1,1
7,Y Chahal,8.5,39,1,0,0,1,0,1,0,1
8,B Kumar,8.5,37,1,0,0,1,0,1,0,1
9,D Chameera,9.0,31,1,0,0,1,0,0,1,1


In [15]:
sum(f_df2.Credits)

96.0

In [18]:
p2 = sum(f_df2.Points)

In [20]:
df3 = pd.read_csv('ind_sl_t20i_3rd_2022_02_27.csv')
df3.head()

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points
0,3692,38642,S Iyer,9428,IND,2,BAT,88.8,8.5,196.0,2,1,100
1,3692,38642,R Jadeja,587,IND,2,AR,87.6,9.0,127.0,2,1,31
2,3692,38642,S Samson,8271,IND,2,WK,85.5,8.0,59.0,1,1,35
3,3692,38642,D Shanaka,8422,SL,5,AR,82.3,9.0,109.0,2,1,105
4,3692,38642,P Nissanka,13682,SL,5,BAT,82.0,9.5,102.0,1,1,-1


In [21]:
f_df3=fantasy(df3)

In [22]:
sum(f_df3.Credits)

94.5

In [23]:
p3 = sum(f_df3.Points)

In [24]:
avg_Points_per_Team=np.mean([p1,p2,p3])

In [25]:
avg_Points_per_Team

630.0